# Pattern Validation

This notebook validates that the Python implementation matches the Pine Script patterns exactly.

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# Load the integrated framework
%run Integrated_Strategy_Backtesting_Framework.ipynb

## Pine Script Pattern Definitions (from Algo Strategy Builder.txt)

### Sacudida Long:
```pinescript
sacudida_long_condition() =>
    vela2_bajista        = close[1] < open[1]
    vela2_rompe_minimo   = low[1]   < low[2]
    vela3_alcista        = close    > open
    vela3_confirmacion   = close    > low[2]
    vela2_bajista and vela2_rompe_minimo and vela3_alcista and vela3_confirmacion
```

### Sacudida Short:
```pinescript
sacudida_short_condition() =>
    vela2_alcista        = close[1] > open[1]
    vela2_rompe_maximo   = high[1]  > high[2]
    vela3_bajista        = close    < open
    vela3_confirmacion   = close    < high[2]
    vela2_alcista and vela2_rompe_maximo and vela3_bajista and vela3_confirmacion
```

### Envolvente Long (bullEngulf):
```pinescript
bullEngulf() =>
    VelaAlcista = close > open
    VelaBajistaPrev = close[1] < open[1]
    cierra_sobre_ap1 = close >= open[1]
    abre_bajo_c1     = open  <= close[1]
    VelaAlcista and VelaBajistaPrev and cierra_sobre_ap1 and abre_bajo_c1
```

### Envolvente Short (bearEngulf):
```pinescript
bearEngulf() =>
    VelaBajista = close < open
    VelaAlcistaPrev = close[1] > open[1]
    cierra_bajo_ap1  = close <= open[1]
    abre_sobre_c1    = open  >= close[1]
    VelaBajista and VelaAlcistaPrev and cierra_bajo_ap1 and abre_sobre_c1
```

### Volumen Climático:
```pinescript
volMA20        = ta.sma(volume, 20)
volClimatico   = not na(volMA20) and volume > volMA20 * 1.75  
clim_long_raw  = volClimatico and close > open            
clim_short_raw = volClimatico and close < open
```

## Validation Test: Pattern Detection

In [ ]:
# Download test data
df_test = yf.download('AAPL', start='2023-01-01', end='2023-12-31')
df_test = ocpVolumeSma(df_test, 20)

print(f"Test data: {len(df_test)} bars")
print(f"Date range: {df_test.index[0]} to {df_test.index[-1]}")

### Test 1: Sacudida Pattern

In [ ]:
# Detect Sacudida Long signals
sacudida_long_signals = PatternLibrary.sacudida_long(df_test)
sacudida_short_signals = PatternLibrary.sacudida_short(df_test)

print(f"Sacudida Long signals detected: {sacudida_long_signals.sum()}")
print(f"Sacudida Short signals detected: {sacudida_short_signals.sum()}")

# Show first few occurrences
if sacudida_long_signals.sum() > 0:
    print("\nFirst 5 Sacudida Long occurrences:")
    long_dates = df_test[sacudida_long_signals].index[:5]
    for date in long_dates:
        idx = df_test.index.get_loc(date)
        print(f"\nDate: {date.date()}")
        print(f"  Bar[i-2]: O={df_test['Open'].iloc[idx-2]:.2f}, H={df_test['High'].iloc[idx-2]:.2f}, L={df_test['Low'].iloc[idx-2]:.2f}, C={df_test['Close'].iloc[idx-2]:.2f}")
        print(f"  Bar[i-1]: O={df_test['Open'].iloc[idx-1]:.2f}, H={df_test['High'].iloc[idx-1]:.2f}, L={df_test['Low'].iloc[idx-1]:.2f}, C={df_test['Close'].iloc[idx-1]:.2f} <- Bearish, breaks low")
        print(f"  Bar[i]  : O={df_test['Open'].iloc[idx]:.2f}, H={df_test['High'].iloc[idx]:.2f}, L={df_test['Low'].iloc[idx]:.2f}, C={df_test['Close'].iloc[idx]:.2f} <- Bullish, confirms")
        
        # Verify conditions
        vela2_bajista = df_test['Close'].iloc[idx-1] < df_test['Open'].iloc[idx-1]
        vela2_rompe_minimo = df_test['Low'].iloc[idx-1] < df_test['Low'].iloc[idx-2]
        vela3_alcista = df_test['Close'].iloc[idx] > df_test['Open'].iloc[idx]
        vela3_confirmacion = df_test['Close'].iloc[idx] > df_test['Low'].iloc[idx-2]
        
        print(f"  Conditions: vela2_bajista={vela2_bajista}, vela2_rompe_minimo={vela2_rompe_minimo}, vela3_alcista={vela3_alcista}, vela3_confirmacion={vela3_confirmacion}")

### Test 2: Envolvente Pattern

In [ ]:
# Detect Envolvente signals
envolvente_long_signals = PatternLibrary.envolvente_long(df_test)
envolvente_short_signals = PatternLibrary.envolvente_short(df_test)

print(f"Envolvente Long (Bullish Engulfing) signals detected: {envolvente_long_signals.sum()}")
print(f"Envolvente Short (Bearish Engulfing) signals detected: {envolvente_short_signals.sum()}")

# Show first few occurrences
if envolvente_long_signals.sum() > 0:
    print("\nFirst 5 Bullish Engulfing occurrences:")
    long_dates = df_test[envolvente_long_signals].index[:5]
    for date in long_dates:
        idx = df_test.index.get_loc(date)
        print(f"\nDate: {date.date()}")
        print(f"  Bar[i-1]: O={df_test['Open'].iloc[idx-1]:.2f}, C={df_test['Close'].iloc[idx-1]:.2f} <- Bearish")
        print(f"  Bar[i]  : O={df_test['Open'].iloc[idx]:.2f}, C={df_test['Close'].iloc[idx]:.2f} <- Bullish, engulfs")
        
        # Verify conditions
        vela_alcista = df_test['Close'].iloc[idx] > df_test['Open'].iloc[idx]
        vela_bajista_prev = df_test['Close'].iloc[idx-1] < df_test['Open'].iloc[idx-1]
        cierra_sobre = df_test['Close'].iloc[idx] >= df_test['Open'].iloc[idx-1]
        abre_bajo = df_test['Open'].iloc[idx] <= df_test['Close'].iloc[idx-1]
        
        print(f"  Conditions: vela_alcista={vela_alcista}, vela_bajista_prev={vela_bajista_prev}, cierra_sobre_ap1={cierra_sobre}, abre_bajo_c1={abre_bajo}")

### Test 3: Volumen Climático Pattern

In [ ]:
# Detect Volumen Climático signals
volumen_long_signals = PatternLibrary.volumen_climatico_long(df_test, vol_multiplier=1.75, vol_period=20)
volumen_short_signals = PatternLibrary.volumen_climatico_short(df_test, vol_multiplier=1.75, vol_period=20)

print(f"Volumen Climático Long signals detected: {volumen_long_signals.sum()}")
print(f"Volumen Climático Short signals detected: {volumen_short_signals.sum()}")

# Show first few occurrences
if volumen_long_signals.sum() > 0:
    print("\nFirst 5 Climatic Volume Long occurrences:")
    long_dates = df_test[volumen_long_signals].index[:5]
    for date in long_dates:
        idx = df_test.index.get_loc(date)
        vol = df_test['Volume'].iloc[idx]
        vol_sma = df_test['volSma20'].iloc[idx]
        ratio = vol / vol_sma if vol_sma > 0 else 0
        
        print(f"\nDate: {date.date()}")
        print(f"  Volume: {vol:,.0f}")
        print(f"  Volume SMA(20): {vol_sma:,.0f}")
        print(f"  Ratio: {ratio:.2f}x (threshold: 1.75x)")
        print(f"  Bar: O={df_test['Open'].iloc[idx]:.2f}, C={df_test['Close'].iloc[idx]:.2f} <- Bullish candle")
        
        # Verify conditions
        vol_climatico = vol > vol_sma * 1.75
        bullish = df_test['Close'].iloc[idx] > df_test['Open'].iloc[idx]
        
        print(f"  Conditions: vol_climatico={vol_climatico}, bullish_candle={bullish}")

### Visual Validation: Chart with Patterns Marked

In [ ]:
# Create visual validation chart
fig, axes = plt.subplots(4, 1, figsize=(15, 12))

# Plot price
axes[0].plot(df_test.index, df_test['Close'], label='Close Price', linewidth=1, color='black')
axes[0].scatter(df_test[sacudida_long_signals].index, df_test[sacudida_long_signals]['Close'], 
                color='green', marker='^', s=100, label='Sacudida Long', zorder=5)
axes[0].scatter(df_test[sacudida_short_signals].index, df_test[sacudida_short_signals]['Close'], 
                color='red', marker='v', s=100, label='Sacudida Short', zorder=5)
axes[0].set_title('Sacudida Pattern Detection', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot Envolvente
axes[1].plot(df_test.index, df_test['Close'], label='Close Price', linewidth=1, color='black')
axes[1].scatter(df_test[envolvente_long_signals].index, df_test[envolvente_long_signals]['Close'], 
                color='green', marker='^', s=100, label='Bullish Engulfing', zorder=5)
axes[1].scatter(df_test[envolvente_short_signals].index, df_test[envolvente_short_signals]['Close'], 
                color='red', marker='v', s=100, label='Bearish Engulfing', zorder=5)
axes[1].set_title('Envolvente (Engulfing) Pattern Detection', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot Volumen Climático
axes[2].plot(df_test.index, df_test['Close'], label='Close Price', linewidth=1, color='black')
axes[2].scatter(df_test[volumen_long_signals].index, df_test[volumen_long_signals]['Close'], 
                color='green', marker='^', s=100, label='Climatic Vol Long', zorder=5)
axes[2].scatter(df_test[volumen_short_signals].index, df_test[volumen_short_signals]['Close'], 
                color='red', marker='v', s=100, label='Climatic Vol Short', zorder=5)
axes[2].set_title('Volumen Climático Pattern Detection', fontsize=14, fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

# Plot Volume with threshold
axes[3].bar(df_test.index, df_test['Volume'], alpha=0.3, color='blue')
axes[3].plot(df_test.index, df_test['volSma20'], label='Volume SMA(20)', linewidth=2, color='orange')
axes[3].plot(df_test.index, df_test['volSma20'] * 1.75, label='1.75x Threshold', 
             linewidth=2, color='red', linestyle='--')
axes[3].set_title('Volume Analysis', fontsize=14, fontweight='bold')
axes[3].legend()
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("VALIDATION SUMMARY")
print("="*60)
print(f"Sacudida Long:        {sacudida_long_signals.sum()} signals")
print(f"Sacudida Short:       {sacudida_short_signals.sum()} signals")
print(f"Envolvente Long:      {envolvente_long_signals.sum()} signals")
print(f"Envolvente Short:     {envolvente_short_signals.sum()} signals")
print(f"Volumen Climático L:  {volumen_long_signals.sum()} signals")
print(f"Volumen Climático S:  {volumen_short_signals.sum()} signals")
print("="*60)

## Code Comparison: Pine Script vs Python

### Sacudida Long Implementation Match:

**Pine Script:**
```pinescript
vela2_bajista        = close[1] < open[1]      // Previous bar is bearish
vela2_rompe_minimo   = low[1]   < low[2]       // Previous bar breaks the low 2 bars ago
vela3_alcista        = close    > open         // Current bar is bullish
vela3_confirmacion   = close    > low[2]       // Current bar closes above low 2 bars ago
```

**Python:**
```python
vela2_bajista = df['Close'].iloc[i-1] < df['Open'].iloc[i-1]      # Previous bar is bearish
vela2_rompe_minimo = df['Low'].iloc[i-1] < df['Low'].iloc[i-2]    # Previous bar breaks the low 2 bars ago
vela3_alcista = df['Close'].iloc[i] > df['Open'].iloc[i]          # Current bar is bullish
vela3_confirmacion = df['Close'].iloc[i] > df['Low'].iloc[i-2]    # Current bar closes above low 2 bars ago
```

✅ **EXACT MATCH**

---

### Indexing Verification:

| Pine Script | Python iloc | Meaning |
|-------------|-------------|----------|
| `close[2]`  | `iloc[i-2]` | 2 bars ago |
| `close[1]`  | `iloc[i-1]` | Previous bar |
| `close`     | `iloc[i]`   | Current bar |

✅ **INDEXING CORRECT**